In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay, balanced_accuracy_score
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.optimizers import Adam
#from keras.backend import expand_dims
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential
from keras.constraints import MaxNorm
from keras.layers import Activation, Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization, SpatialDropout1D
import os


2025-03-03 15:30:33.866301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-03 15:30:34.007867: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 15:30:34.043895: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 15:30:34.665599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

2025-03-03 15:30:35.568538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-03 15:30:36.204665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14120 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:c3:00.0, compute capability: 8.6


In [ ]:
def columnas_bacterias_fun():
    vocales = ['a','e','i','o','u']
    columnas_bacterias = []
    for i in vocales:
        for j in df.columns:
            if i in j:
                columnas_bacterias.append(j)
    columnas_bacterias = list(set(columnas_bacterias))
    return columnas_bacterias


files_list = os.listdir('SetDatos/')
for archivo in files_list:
    print(archivo)
    df = pd.read_csv('SetDatos/'+archivo)
    df = df.drop(columns=['code','species'])
    df.dropna(axis=0, how="any", inplace=True)
    columnas_bacterias = columnas_bacterias_fun()
    for bacteria in columnas_bacterias:
        try:
            print('Archivo:',archivo,'Bacteria:',bacteria)
            columnas_bacterias_sin_bacteria = [b for b in columnas_bacterias if b != bacteria]
            df_bacteria = df.drop(columns = columnas_bacterias_sin_bacteria)
            bacteria = df_bacteria.columns[-1]
            X = df_bacteria.iloc[:, 0:-2].values  # variables independientes (espectros de masa)
            y = df_bacteria.iloc[:, -1].values    # variable dependientes (resistencia a ciprofloxacin)
            X = np.asarray(X).astype(np.float32)
            y = np.asarray(y).astype(np.float32)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify=y)
            

            #entrenamiento sin oversampling.

            scaler=Normalizer(norm='max')
            sc_X = scaler
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)

            sample_size = X_train.shape[0] # numero de muestras en el set de datos
            time_steps  = X_train.shape[1] # numero de atributos en el set de datos
            input_dimension = 1            #

            X_train_reshaped = X_train.reshape(sample_size,time_steps,input_dimension)
            X_test_reshaped = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001)
            early_st = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

            n_timesteps = X_train_reshaped.shape[1] #
            n_features  = X_train_reshaped.shape[2] #


            model = Sequential(name="Modelo_s_aureus_ciprofloxacin")
            init_mode = 'normal'
            model.add(Conv1D(filters=(64), kernel_size=(17), input_shape = (n_timesteps,n_features), name='Conv_1'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(MaxPooling1D(pool_size=2, name="MaxPooling1D_1"))

            model.add(Conv1D(filters=(128), kernel_size=(9),kernel_initializer=init_mode, kernel_regularizer=regularizers.l2(0.0001),  name='Conv_2'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(MaxPooling1D(pool_size=2, name="MaxPooling1D_2"))

            model.add(Conv1D(filters=(256), kernel_size=(5),kernel_initializer=init_mode,kernel_regularizer=regularizers.l2(0.0001),   name='Conv_3'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(MaxPooling1D(pool_size=2, name="MaxPooling1D_3"))

            model.add(Conv1D(filters=(256), kernel_size=(5),kernel_initializer=init_mode, kernel_regularizer=regularizers.l2(0.0001),   name='Conv_4'))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(MaxPooling1D(pool_size=2, name="MaxPooling1D_4"))

            model.add(Flatten())
            model.add(Dropout(0.65))
            model.add(Dense(256, activation='relu',kernel_initializer=init_mode, kernel_regularizer=regularizers.l2(0.0001), name="fully_connected_0"))
            model.add(Dense(64, activation='relu',kernel_initializer=init_mode, kernel_regularizer=regularizers.l2(0.0001), name="fully_connected_1"))
            model.add(Dense(64, activation='relu',kernel_initializer=init_mode, kernel_regularizer=regularizers.l2(0.0001),  name="fully_connected_2"))
            model.add(Dense(n_features, activation='sigmoid', name="OUT_Layer"))

            model.compile(optimizer = Adam(learning_rate=0.0001), loss = 'binary_crossentropy',  metrics=METRICS)
            model.summary()
            history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=10, verbose=1, validation_split=0.1, callbacks=[reduce_lr,early_st])

            y_pred  = model.predict(X_test_reshaped)
            y_pred = (y_pred>0.5)
            cm = confusion_matrix(y_test, y_pred)
            # Abre el archivo en modo apéndice
            with open('resultados/resultados.txt', 'a') as archivo_:
                # Redirige la salida estándar al archivo
                print('-----------------------------------------------------\n\n','nombre de archivo:', archivo, '\nBacteria:', bacteria, '\n\nconfusion_matrix:\n', cm,"\n\nbalanced acuracy:", balanced_accuracy_score(y_test, y_pred), file=archivo_)
                target_names=["0","1"]
                print('\n\n',classification_report(y_test, y_pred, target_names=target_names), file=archivo_)

                train_predictions_baseline = model.predict(X_train_reshaped, batch_size=10)
                test_predictions_baseline = model.predict(X_test_reshaped, batch_size=10)
                print('\n\n')
                baseline_results = model.evaluate(X_test_reshaped, y_test, verbose=0)
                for name, value in zip(model.metrics_names, baseline_results):
                    print(name, ': ', value, file=archivo_)
        except ValueError as e:
            with open('resultados/resultados.txt', 'a') as archivo_:
                print("Error:",e,file = archivo_)



e_coli_driams_b_2000_20000Da_v2 (1).csv
Archivo: e_coli_driams_b_2000_20000Da_v2 (1).csv Bacteria: Cefepime
Model: "Modelo_s_aureus_ciprofloxacin"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv1D)             (None, 5983, 64)          1152      
                                                                 
 batch_normalization (BatchN  (None, 5983, 64)         256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 5983, 64)          0         
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 2991, 64)         0         
 D)                                                              
                                                                 
 Conv_2 (Conv1D)             (None, 2983, 128)         73856 

2025-03-03 15:30:39.766904: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8907
2025-03-03 15:30:39.792178: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-03-03 15:30:40.725245: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


16/16 [==============================] - 5s 97ms/step - loss: 7.2703 - tp: 4.0000 - fp: 10.0000 - tn: 113.0000 - fn: 26.0000 - accuracy: 0.7647 - precision: 0.2857 - recall: 0.1333 - auc: 0.4518 - prc: 0.2232 - val_loss: 6.7946 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 13.0000 - val_fn: 4.0000 - val_accuracy: 0.7647 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.2174 - lr: 1.0000e-04
Epoch 2/100
16/16 [==============================] - 1s 37ms/step - loss: 6.7102 - tp: 2.0000 - fp: 15.0000 - tn: 108.0000 - fn: 28.0000 - accuracy: 0.7190 - precision: 0.1176 - recall: 0.0667 - auc: 0.5331 - prc: 0.2009 - val_loss: 6.7042 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 13.0000 - val_fn: 4.0000 - val_accuracy: 0.7647 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.2174 - lr: 1.0000e-04
Epoch 3/100
16/16 [==============================] - 1s 35ms/step - loss: 6.6096 - tp: 7.0000 - fp: 10.0000 - tn: 113.00

/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5/5 [==============================] - 0s 5ms/step



Archivo: e_coli_driams_b_2000_20000Da_v2 (1).csv Bacteria: Ceftriaxone
Model: "Modelo_s_aureus_ciprofloxacin"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv1D)             (None, 5983, 64)          1152      
                                                                 
 batch_normalization_4 (Batc  (None, 5983, 64)         256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 5983, 64)          0         
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 2991, 64)         0         
 D)                                                              
                                                                 
 Conv_2 (Conv1D)             (None, 2983, 12

/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


3/3 [==============================] - 0s 5ms/step



Archivo: k_pneumoniae_driams_b_2000_20000Da_v2 (1).csv Bacteria: Ceftriaxone
Model: "Modelo_s_aureus_ciprofloxacin"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv1D)             (None, 5983, 64)          1152      
                                                                 
 batch_normalization_20 (Bat  (None, 5983, 64)         256       
 chNormalization)                                                
                                                                 
 activation_20 (Activation)  (None, 5983, 64)          0         
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 2991, 64)         0         
 D)                                                              
                                                                 
 Conv_2 (Conv1D)             (None, 29

/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hpinto/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))





Archivo: k_pneumoniae_driams_b_2000_20000Da_v2 (1).csv Bacteria: Ciprofloxacin
Model: "Modelo_s_aureus_ciprofloxacin"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv_1 (Conv1D)             (None, 5983, 64)          1152      
                                                                 
 batch_normalization_24 (Bat  (None, 5983, 64)         256       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 5983, 64)          0         
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 2991, 64)         0         
 D)                                                              
                                                                 
 Conv_2 (Conv1D)             (None, 2983, 128)         73856     
                     